In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras as keras
from keras import layers as layers

In [3]:
import os, timeit
from skimage.filters import threshold_otsu
import numpy as np
from math import inf as inf

In [4]:
from spectral.io import envi as envi
from spectral import imshow

In [5]:
from sklearn.decomposition import IncrementalPCA

In [6]:
import sys

In [7]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [8]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [9]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.5
TOTAL_IMAGE_COUNT = 8
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 4

FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 50
IMAGE_HEIGHT = 50

HSI_CORRECTION = True

FILTER = ""

DIMENSION_REDUCTION = True
NUM_OF_BANDS = 3

DATA_AUGMENTATION = True

In [10]:
DARK_CORRECTION_2 = DATA_DIRECTORY+"Dark Correction_2_34"
DARK_CORRECTION_4 = DATA_DIRECTORY+"Dark Correction_4_58"
RESPONSE_CORRECTION_2 = DATA_DIRECTORY+"Response Correction_2_35"
RESPONSE_CORRECTION_4 = DATA_DIRECTORY+"Response Correction_4_59"

In [11]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [12]:
def exactPathHDR(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil.hdr"

def exactPathBIL(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil"

In [13]:
def correction(img,idx):
    return img

In [14]:
def apply_filters(img):
    if FILTER == "":
        return img
    else:
        return img

In [15]:
def getROI(img, band_number):
    img_band = img.read_band(band_number)
    threshold = threshold_otsu(img_band)
    roi=[]
    for x in range(img_band.shape[0]):
        a=[]
        for y in range(img_band.shape[1]):
            if img_band[x][y]>threshold:
                a.append(1)
            else:
                a.append(0)
        roi.append(a)
    return roi

In [16]:
#Returns range for x and y from where we have to crop images
def getRangeXandY(img,band_number):
    img_band = img.read_band(band_number)
    roi = getROI(img,band_number)
    xmin = inf
    xmax = 0
    ymin = inf
    ymax = 0
    for x in range(img_band.shape[0]):
        for y in range(img_band.shape[1]):
            if roi[x][y]==1:
                if x<xmin:
                    xmin=x
                if x>xmax:
                    xmax=x
                if y<ymin:
                    ymin=y
                if y>ymax:
                    ymax=y
    return xmin, xmax, ymin, ymax

In [17]:
def getCroppedImage(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    new_img = img[xmin:xmax, ymin:ymax, :]
    return new_img

In [18]:
def getCroppedROI(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    roi = np.array(getROI(img,band_number))
    roi = roi[xmin:xmax, ymin:ymax]
    return roi   

In [19]:
def getUsefulImage(img,band_number):
    crop_img = getCroppedImage(img,band_number)
    crop_roi = getCroppedROI(img,band_number)
    for x in range(crop_img.shape[2]):
        band = crop_img[:,:,x]
        crop_img[:,:,x] = band*crop_roi
    return crop_img

In [1]:
def preprocessHSI(img, idx, band_number):
    corr_img = img
    if HSI_CORRECTION:
        corr_img = correction(img, idx)
    img = getUsefulImage(corr_img, band_number)
    return img

In [21]:
crop_data = keras.Sequential([
    layers.RandomCrop(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
])
def cropImage(img,band_number):
#     new_img = getUsefulImage(img,band_number)
    cropped_img = crop_data(img)
    return cropped_img

def checkCroppedImage(cropped_img):
    crop_band = cropped_img[:,:,0]
    filled_area_ratio = (np.count_nonzero(crop_band))/(crop_band.shape[0]*crop_band.shape[1])
    if filled_area_ratio > FILLED_AREA_RATIO :
        return True
    else:
        return False

In [22]:
data_augmentation = keras.Sequential([
    layers.RandomRotation(factor=(-0.1, 0.1)),
    layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1,0.1)),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=None)
])

def getAugumentedImage(img):
    augmented_image = data_augmentation(img) 
    return augmented_image

In [23]:
## Dimensional Reduction Method
def DL_Method(HSI, numComponents = NUM_OF_BANDS):
    if DIMENSION_REDUCTION:
        RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
        n_batches = 10
        inc_pca = IncrementalPCA(n_components=numComponents)
        for X_batch in np.array_split(RHSI, n_batches):
            inc_pca.partial_fit(X_batch)
        X_ipca = inc_pca.transform(RHSI)
        RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
        return RHSI
    else:
        return HSI

In [24]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [25]:
#List for all file names in varities
FILES = []
MAX_FILE_NUM = 4
for x in range(1,MAX_FILE_NUM+1):
    FILES.append("B_"+str(x))

In [26]:
#List of all images
images = []
images_label = []
for v in VARIETIES:
    for f in FILES:
        try:
            img = envi.open(exactPathHDR(v,f),exactPathBIL(v,f))
            idx = int(f[-1])
            img = preprocessHSI(img, idx, BAND_NUMBER)
            images.append(img)
            images_label.append(v)
        except:
            pass

In [27]:
train_dataset = []
train_dataset_label = []
test_dataset = []
test_dataset_label = []

In [28]:
tic = start_timer()
for index, img in enumerate(images):
    count = 0
    label = images_label[index]
    while count<IMAGE_COUNT:
        crop_img = cropImage(img,BAND_NUMBER)
        aug_img = crop_img
        if checkCroppedImage(crop_img):
            if DATA_AUGMENTATION:
                aug_img = getAugumentedImage(aug_img)
            aug_img = apply_filters(aug_img)
            aug_img = DL_Method(aug_img[:,:,FIRST_BAND:LAST_BAND+1])
            if count%5 == 0:
                test_dataset.append(aug_img)
                test_dataset_label.append(label)
            else:
                train_dataset.append(aug_img)
                train_dataset_label.append(label)
            count+=1 
            
toc = end_timer()
show_time(tic,toc)

Testing started


C:\Users\ganga\anaconda3\lib\site-packages\sklearn\decomposition\_incremental_pca.py:338: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
C:\Users\ganga\anaconda3\lib\site-packages\sklearn\decomposition\_incremental_pca.py:338: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)


Testing time (s) = 13.961405400000004



In [29]:
train_dataset = np.array(train_dataset)
train_dataset_label = np.array([VARIETIES_CODE[label] for label in train_dataset_label])
test_dataset = np.array(test_dataset)
test_dataset_label = np.array([VARIETIES_CODE[label] for label in test_dataset_label])

In [30]:
results_dir = os.path.join('./dataset')
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [31]:
def dataset_file_name():
    return "./dataset/V"+str(NUM_VARIETIES).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_HSI_CORRECTION_"+str(HSI_CORRECTION)+"_FILTER_"+str(FILTER)+"_DIMENSION_REDUCTION_"+str(DIMENSION_REDUCTION)+"_DATA_AUGMENTATION_"+str(DATA_AUGMENTATION)

In [32]:
DATASET_FILE_NAME = dataset_file_name()
np.save(DATASET_FILE_NAME+"_train_dataset.npy",train_dataset)
np.save(DATASET_FILE_NAME+"_train_dataset_label.npy",train_dataset_label)
np.save(DATASET_FILE_NAME+"_test_dataset.npy",test_dataset)
np.save(DATASET_FILE_NAME+"_test_dataset_label.npy",test_dataset_label)